### via europe code 

In [107]:
asins = ['B0038ZIY9I',
'B0041L70Q6',
'B00509CI38',
'B005F8U502',
'B00B4RT5HW',
'B00WDZ9HVI',
'B01CYMJEAS',
'B01JRXTOIE',
'B01N9LYY90',
'B06ZXZWB8F',
'B072XMZ7J9',
'B073LYWDJH',
'B074MYSX1Y',
'B074P1FX4K',
'B075DR9ZPT',
'B076DS6BT4']


In [8]:
import requests
import json
import pandas as pd
import openpyxl

## rainforest 

In [8]:
# Read the Excel file into a pandas DataFrame
import pandas as pd
df = pd.read_excel('../documents/Taric Code_Test file.xlsx')
df.head()

,ASIN,URL,Name,VE Code,VE Duty %,AI Code,AI Duty %,AI Reasoning,Unnamed: 8
0,NaN,Classification,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,B09RDXKH4V,https://www.amazon.de/gp/product/B09RDXKH4V,Pyatofyy Key - Scooter Bicycle Headset Wrench...,8.203100e+09,0.017,8.204110e+09,0.017,This Taric code is chosen because the product ...,NaN
2,B0BMG63KL4,https://www.amazon.de/dp/B0BMG63KL4?ref=myi_ti...,Colexy Fresh System Food Storage Container,3.924100e+09,0.065,3.924100e+09,0.065,I have chosen this Taric code because the prod...,NaN
3,B09C823KH1,https://www.amazon.de/gp/product/B09C823KH1,Pyatofyy 48 Packs Sign Spacing Screws,7.317008e+09,0.000,8.302420e+09,0.027,I have chosen this Taric code because the prod...,NaN
4,B08GYMPKSQ,https://www.amazon.de/dp/B08GYMPKSQ,TopHGC Hair Dryer Holder,7.616999e+09,0.060,8.302500e+09,0.027,I have chosen this Taric code because the prod...,NaN


In [158]:
asin = 'B076DS6BT4'
asin = 'B0041L70Q6'
asin = 'B0038ZIY9I'# test on one asin 
rainforest_api_key = '2BE30AF36DC7483AA7FE7E9CA64F46FD'
amazon_domain = 'amazon.com'

In [117]:
def scrap_asin (asin ,rainforest_api_key , amazon_domain  ):
    params = {
              'api_key': rainforest_api_key,
              'amazon_domain': amazon_domain,
              'asin': asin,
              'type': 'product'}
    api_result = requests.get('https://api.rainforestapi.com/request', params)
    api_json = api_result.json()
    return api_json 

In [104]:
def extract_features(api_json) : 
    specs = api_json['product']['specifications']
    title = api_json['product']['title']
    columns = [x['name'] for x in specs]
    values = [x['value'] for x in specs]
    df_spec = pd.DataFrame([values], columns=columns)
    df_spec['title'] = title
    return df_spec

In [159]:

api_json_ = scrap_asin (asin ,rainforest_api_key , amazon_domain  )

In [153]:
api_json_['request_info']['success']

True

In [154]:
for key in api_json_['product'].keys() : 
    print(key)

title
search_alias
keywords
keywords_list
asin
link
brand
sell_on_amazon
categories
categories_flat
description
sub_title
rating
rating_breakdown
ratings_total
main_image
images
images_count
images_flat
is_bundle
attributes
top_reviews
buybox_winner
specifications
specifications_flat
color
weight
first_available
dimensions
model_number


In [156]:
api_json['product']['title']

'Dajar Porto Milk Jug 210\xa0ml, Porcelain, White, 11\xa0x 5,7\xa0x 9.8\xa0cm'

In [163]:
api_json_['product']

{'title': 'Mammut Dual Adjust Side Squeeze Buckle',
 'search_alias': {'title': 'All Departments', 'value': 'aps'},
 'keywords': 'Mammut,Dual,Adjust,Side,Squeeze,Buckle',
 'keywords_list': ['Mammut', 'Dual', 'Adjust', 'Side', 'Squeeze', 'Buckle'],
 'asin': 'B0038ZIY9I',
 'link': 'https://www.amazon.com/Mammut-Dual-Adjust-Squeeze-2540-00150-0001-1015/dp/B0038ZIY9I',
 'brand': 'Mammut',
 'has_size_guide': True,
 'categories': [{'name': 'All Departments'},
  {'name': 'Sports & Outdoors',
   'link': 'https://www.amazon.com/sports-outdoors/b/ref=dp_bc_aui_C_1?ie=UTF8&node=3375251',
   'category_id': '3375251'},
  {'name': 'Outdoor Recreation',
   'link': 'https://www.amazon.com/outdoor-recreation/b/ref=dp_bc_aui_C_2?ie=UTF8&node=706814011',
   'category_id': '706814011'},
  {'name': 'Camping & Hiking',
   'link': 'https://www.amazon.com/camping-hiking/b/ref=dp_bc_aui_C_3?ie=UTF8&node=3400371',
   'category_id': '3400371'},
  {'name': 'Backpacks, Bags & Accessories',
   'link': 'https://www.a

In [157]:
extract_features(api_json_)

,Item Weight,Product Dimensions,ASIN,Item model number,Customer Reviews,Is Discontinued By Manufacturer,Date First Available,Color Name,Size,Brand,Color,Item Weight,Item Dimensions LxWxH,title
0,0.704 ounces,8.86 x 3.94 x 1.18 inches,B0041L70Q6,8626P,3.8 out of 5 stars 938 ratings ...,No,"October 3, 2014",Multicoloured,One Size,Sancto,Multicoloured,0.02 Kilograms,8.86 x 3.94 x 1.18 inches,Pipes 145Mm Halloween Novelty Toy Weapons And ...


In [160]:
extract_features(api_json_)

,Package Dimensions,Item model number,Department,Date First Available,Manufacturer,ASIN,title
0,3.39 x 2.91 x 0.24 inches; 0.71 Ounces,2540-00150,unisex-adult,"April 9, 2014",Mammut,B0038ZIY9I,Mammut Dual Adjust Side Squeeze Buckle


### automate on all asin 

In [180]:
%%time
df_all_asin = pd.DataFrame()
for asin in asins : 
    print(asin)
   
    api_json_product = scrap_asin (asin ,rainforest_api_key , amazon_domain )
    if api_json_product['request_info']['success'] == True:
        df_product = extract_features(api_json_product)
        df_all_asin = df_all_asin.append(df_product, ignore_index=True)
    else  : 
        print('cannot scrapp  asin number : ', asin )
        print( 'the message is : '  ,  api_json_product['request_info']['message'])

B0038ZIY9I


<timed exec>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


B0041L70Q6


<timed exec>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [176]:
df_all_asin

,Package Dimensions,Item model number,Department,Date First Available,Manufacturer,ASIN,title,Item Weight,Product Dimensions,ASIN,...,Style,Model Name,Product Dimensions,Item Weight,Manufacturer,ASIN,Item model number,Customer Reviews,Date First Available,title
0,3.39 x 2.91 x 0.24 inches; 0.71 Ounces,2540-00150,unisex-adult,"April 9, 2014",Mammut,B0038ZIY9I,Mammut Dual Adjust Side Squeeze Buckle,0.704 ounces,8.86 x 3.94 x 1.18 inches,B0041L70Q6,...,Modern,PORTO,4.33 x 2.24 x 3.86 inches,7 ounces,Ambition,B076DS6BT4,33961,4.3 out of 5 stars 147 ratings ...,"May 31, 2019","Dajar Porto Milk Jug 210 ml, Porcelain, White,..."
